# Tutorial VIII - Police Districting

Applied Optimization with Julia

# Introduction

Imagine you’re the lead consultant for Hamburg’s police force
modernization project. The city is facing increasing response times in
certain districts, and the Chief of Police has hired you to optimize
their district boundaries. Your mission: redesign Hamburg’s police
districts to ensure rapid response times while meeting several critical
operational requirements.

Key Objectives: - Minimize average response time to incidents - Ensure
every area has adequate police coverage - Enable efficient backup
support between districts - Account for real-world constraints like
traffic patterns

Throughout this tutorial, you’ll build this solution step-by-step,
starting with a basic model and gradually adding real-world constraints
that police departments face daily.

# 1. Modelling the P-Median Problem

Your first task is to fix a prototype model. It contains 6 critical
errors that are causing the model to fail. As the lead consultant, you
need to:

1.  Identify and fix these errors
2.  Document why each fix was necessary in a comment
3.  Validate that your solution makes sense

In this task it is **not yet necessary** to include contiguity and
compactness constraints! If the model takes too long on your machine,
you can set the **relative gap to 10% or decrease the time limit**.

To execute the code, several files are provided, which contain the
Euclidean distance between the BAs, the driving time between BAs, the
number of incidents per BA, and the hexagonal shapes (a `.shp` file and
a `.dbf` file - you need both!).

## Loading Data and Packages

We start by loading the new `Shapefile` package. **No mistakes here!**

In [1]:
import Pkg
Pkg.add("Shapefile")

We then load the required packages. If you have not installed them yet,
you can do so by modifying the code chunk above to include the missing
packages. **Still no mistakes!**

In [1]:
using JuMP, HiGHS
using DelimitedFiles
using Shapefile
using DataFrames
using Plots

We then define the number of departments, load the data into scope and
define the number of departments and the weighted driving times. Make
sure to use the correct path to the data files - it should be relative
to the location notebook file in a folder called `data`. **Again, no
mistakes!**

In [1]:
# Load the data into scope
file_directory = "$(@__DIR__)/data"
euclidianDistances = readdlm("$file_directory/euclidianDistances0510.csv",'\t')
drivingTimes = readdlm("$file_directory/drivingTimes0510.csv",'\t')
incidentWeights = vec(readdlm("$file_directory/incidentWeights0510.csv",'\t'))

# Load the Shapefile for plotting
hexshape = DataFrame(Shapefile.Table("$file_directory/grid0510.shp"))
sort!(hexshape, :id)

# Define the number of departments
p = 10

# Define the weighted driving times
weightedDriving = drivingTimes .* transpose(incidentWeights)

510×510 Matrix{Float64}:
   10.0    88.66    734.08   484.88   327.9  …  0.0  3520.26  4026.38  4456.5
   68.2    13.0     352.16   306.82   235.8     0.0  3468.6   3968.42  4393.5
  118.4    73.84     62.0    164.72   162.3     0.0  3376.35  3864.92  4280.5
  167.2   137.54    352.16    29.0     78.9     0.0  3343.96  3828.58  4241.0
  218.6   204.36    670.84   152.54    15.0     0.0  3314.44  3786.72  4205.0
  279.2   283.14   1046.56   328.28    90.9  …  0.0  3343.14  3818.92  4240.5
  336.8   358.02   1403.68   495.32   177.3     0.0  3272.21  3726.46  4153.5
  388.6   425.36   1724.84   645.54   255.0     0.0  3234.08  3651.02  4092.0
  450.4   505.7    2108.0    824.18   347.4     0.0  3184.06  3594.9   4031.5
  516.6   591.76   2518.44  1016.74   447.0     0.0  3152.49  3559.48  3993.0
    ⋮                                        ⋱                         
 1688.6  2164.5   10042.8   4651.6   2392.5     0.0   835.99  1015.68  1227.5
 1865.2  2392.0   11127.8   5159.68  2647.2  

## Fixing the Mistakes

From here on, the code contains **6 mistakes** you need to correct in
order to solve the model. **Mistakes can be anything so be careful!**

In [1]:
## Prepare the model instance
pMedianModel = Model(HiGHS.Optimizer)
set_attribute(pMedianModel, "presolve", "on")
set_attribute(pMedianModel, "time_limit", 120.0)
set_attribute(pMedianModel, "mip_rel_gap", 0.0)

## Define the range of the problem instance
rangeBAs = 1:size(incidentWeights,1)
rangeDepartments = unique(rand(1:size(incidentWeights,1), 100))  # Ensure unique departments

## Define variable
@variable(pMedianModel, X[i = rangeDepartments,j = rangeBAs], Bin)

## Define objective function
@objective(pMedianModel, Min, 
    sum(weightedDriving[i,j]* X[i,j] for i in rangeDepartments, j in rangeBAs)
    )

## Define the constraints
@constraint(pMedianModel, 
    eachAllocated[j=rangeBAs], 
    sum(X[i,j] for i in rangeDepartments) == 1
    )

@constraint(pMedianModel,
    pLocations, 
    sum(X[i,i] for i in rangeDepartments) == p
    )

@constraint(pMedianModel, 
    departmentNecessary[i=rangeDepartments,j=rangeBAs],
    X[i,j] <= X[i,i]
    )

## Start optimization
optimize!(pMedianModel)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e-01, 4e+04]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+01]
Presolving model
45303 rows, 44880 cols, 134552 nonzeros  0s
45303 rows, 44880 cols, 134552 nonzeros  0s

Solving MIP model with:
   45303 rows
   44880 cols (44880 binary, 0 integer, 0 implied int., 0 continuous)
   134552 nonzeros
MIP-Timing:        0.23 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InL

We then check the solution. **No mistakes here, as all mistakes are in
the code in the cell above!**

In [1]:
# Check solution
function print_model_status(model)
    begin
        println()
    if termination_status(pMedianModel) == OPTIMAL
        println("Great, the solution is optimal.")
        println("The relative gap is $(relative_gap(pMedianModel))")
        println("The solve time (in seconds) is $(solve_time(pMedianModel))")
    elseif termination_status(pMedianModel) == TIME_LIMIT && has_values(pMedianModel)
        println("Solution is suboptimal due to a time limit, but a primal solution is available") 
    else
        error("The model was not solved correctly.") 
    end
        println("The objective value is ", objective_value(pMedianModel)) 
    end
end

print_model_status(pMedianModel)


Great, the solution is optimal.
The relative gap is 0.0
The solve time (in seconds) is 3.0086878342553973
The objective value is 332792.29

In [1]:
@assert termination_status(pMedianModel) == OPTIMAL || (termination_status(pMedianModel) == TIME_LIMIT && has_values(pMedianModel)) "Unfortunate, the model was not solved correctly. Have you corrected all mistakes?"
println("Great, the model was solved correctly.")


Great, the model was solved correctly.

The following code then builds and uses a function to plot the results.

In [1]:
function visualize_departments(hexshape, X, p)
    # Convert solution matrix to regular Matrix
    allAssignments = Matrix(value.(X))
    
    # Find assignments where value > 0.5 (accounting for potential floating-point imprecision)
    assignments = findall(allAssignments .> 0.5)
    
    # Create copy of hexshape to avoid modifying original
    plot_data = copy(hexshape)
    
    # Assign departments and initialize colors
    plot_data.department = rangeDepartments[map(x->x[1], assignments)]
    plot_data.color = fill(RGB(0/255, 0/255, 0/255), nrow(plot_data))
    
    # Get unique department locations and create color mapping
    department_locations = unique(rangeDepartments[map(x->x[1], assignments)])
    color_palette = cgrad(:Pastel1_9, p, categorical=true)
    color_dict = Dict(department_locations[i] => color_palette[i] for i in 1:p)
    
    # Color non-department locations
    for hex in eachrow(plot_data)
        if hex.id in department_locations
            hex.color = RGB(0/255, 0/255, 0/255)
        else
            hex.color = color_dict[hex.department]
        end
    end
    
    # Create and return plot
    return plot(
        plot_data.geometry,
        color=plot_data.color',
        legend=false,
        axis=false,
        ticks=false,
        size=(800,450)
    )
end

# Plot the results
plot_area = visualize_departments(hexshape, X, p)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2353.77,977.854 2382.21,925.951

# 2. Ensuring District Connectivity

The Police Chief has identified a critical flaw in the initial model:
some police units would need to drive through other districts to reach
parts of their own district! This creates jurisdictional issues and
slower response times.

Your challenge:

-   Implement contiguity constraints to ensure each district is fully
    connected
-   Compare response times before and after adding these constraints
-   Visualize the impact of your changes on the district map

## Conditional Constraints

Before we start, we will quickly repeat some basic concepts on
constraints and conditions in JuMP. You can add conditions to
constraints by using the `;` operator. This is useful if you want to add
a constraint only under certain conditions. In the example below, the
constraint is only active if the Euclidean distance between two BAs is
less than 1.5.

In [0]:
@constraint(model, 
    conditionalConstraint[
        i=rangeDepartments,
        j=rangeBAs; 
        euclidianDistances[i,j] < 1.5
        ],
    X[i,j] == 1
    )

Furthermore,we can use conditions within constraints by using the `for`
keyword. For example, in the constraint below, the sum is only taken
over the BAs that are within 1.5 units of BA $i$.

In [0]:
@constraint(model, 
    conditionalConstraint2[i=rangeDepartments],
    sum(X[i,j] for j in rangeBAs if euclidianDistances[i,j] < 1.5) >= 1
    )

## Extending the Model

Now, we can start to extend the model. Add the contiguity constraint
from the lecture to the model.

> **Important**
>
> Take a careful look at the Euclidean distances, as you can use them to
> determine if two BAs are adjacent to each other. If the distance
> between two BAs is less than 1.5, then the BAs are adjacent to each
> other. You can use this information to define the new constraint.

In [1]:
# YOUR CODE BELOW
@constraint(pMedianModel, 
    contiguity[i=rangeDepartments,j=rangeBAs; euclidianDistances[i,j] > 1.5],
    X[i,j] <= sum(X[i,v] for v in rangeBAs if euclidianDistances[v,j] < 1.5 && v != j && euclidianDistances[i,v] < euclidianDistances[i,j])
    )


JuMP.Containers.SparseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 2, Tuple{Int64, Int64}} with 44306 entries:
  [406, 1  ]  =  contiguity[406,1] : X[406,1] - X[406,16] ≤ 0
  [406, 2  ]  =  contiguity[406,2] : -X[406,1] + X[406,2] - X[406,16] - X[406,17] ≤ 0
  [406, 3  ]  =  contiguity[406,3] : -X[406,2] + X[406,3] - X[406,17] - X[406,18] ≤ 0
  [406, 4  ]  =  contiguity[406,4] : -X[406,3] + X[406,4] - X[406,18] - X[406,19] ≤ 0
  [406, 5  ]  =  contiguity[406,5] : -X[406,4] + X[406,5] - X[406,19] - X[406,20] ≤ 0
  [406, 6  ]  =  contiguity[406,6] : -X[406,5] + X[406,6] - X[406,20] - X[406,21] ≤ 0
  [406, 7  ]  =  contiguity[406,7] : -X[406,6] + X[406,7] - X[406,21] - X[406,22] ≤ 0
  [327, 503]  =  contiguity[327,503] : -X[327,488] - X[327,489] + X[327,503] - X[327,504] ≤ 0
              ⋮
  [327, 504]  =  contiguity[327,504] : -X[327,489] - X[327,490] + X[327,504] - X[32

Solve the model again, this time with the contiguity constraint.

In [1]:
# YOUR CODE BELOW
optimize!(pMedianModel)

Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e-01, 4e+04]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+01]
Presolving model
89609 rows, 44880 cols, 303775 nonzeros  0s
89609 rows, 44880 cols, 303775 nonzeros  0s

Solving MIP model with:
   89609 rows
   44880 cols (44880 binary, 0 integer, 0 implied int., 0 continuous)
   303775 nonzeros
MIP-Timing:         0.5 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf     

The following code prints the model status and visualizes the districts.
If your implementation is correct, the districts should be contiguous
and the model should have reached optimality or found a feasible
solution before hitting the time limit.

In [1]:
print_model_status(pMedianModel)
display(visualize_departments(hexshape, X, p))


Great, the solution is optimal.
The relative gap is 0.0
The solve time (in seconds) is 4.10600404208526
The objective value is 332792.29

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2353.77,977.854 2382.21,925.951

## Compute the Gap

Based on your results, what is the gap between the solution in the
previous task and this task? Write a comment answering the question in
cell below. You can also use the cell, to compute the gap based on the
objective values.

In [1]:
gap = (objective_value(pMedianModel) - 326607.39) / 326607.39
println("The gap is $(gap)%.")
#=



=#

The gap is 0.01893680360386201%.

> **Tip**
>
> If you computer cannot determine the optimal solution, you can just
> use the best solutions you found after both runs to compute the gap.

# 3. Emergency Response Time Guarantees

Hamburg’s City Council has mandated that high-priority emergencies must
receive a response within 20 minutes. Your previous model doesn’t
guarantee this! Your task: Implement maximum response time constraints.

Extend your model by an additional parameter `max_driving_time = 20` to
ensure that no allocation with a driving time $d_{i,j}$ higher than
`max_driving_time` minutes from $i$ to $j$ is possible.

In [1]:
# YOUR CODE BELOW
max_driving_time = 20.0

@constraint(pMedianModel, 
    maxDrivingTime[i=rangeDepartments,j=rangeBAs; drivingTimes[i,j] >= max_driving_time],
    X[i,j] == 0
    )

JuMP.Containers.SparseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}, 2, Tuple{Int64, Int64}} with 34059 entries:
  [406, 1  ]  =  maxDrivingTime[406,1] : X[406,1] = 0
  [406, 2  ]  =  maxDrivingTime[406,2] : X[406,2] = 0
  [406, 3  ]  =  maxDrivingTime[406,3] : X[406,3] = 0
  [406, 4  ]  =  maxDrivingTime[406,4] : X[406,4] = 0
  [406, 5  ]  =  maxDrivingTime[406,5] : X[406,5] = 0
  [406, 6  ]  =  maxDrivingTime[406,6] : X[406,6] = 0
  [406, 7  ]  =  maxDrivingTime[406,7] : X[406,7] = 0
  [327, 503]  =  maxDrivingTime[327,503] : X[327,503] = 0
              ⋮
  [327, 504]  =  maxDrivingTime[327,504] : X[327,504] = 0
  [327, 505]  =  maxDrivingTime[327,505] : X[327,505] = 0
  [327, 506]  =  maxDrivingTime[327,506] : X[327,506] = 0
  [327, 507]  =  maxDrivingTime[327,507] : X[327,507] = 0
  [327, 508]  =  maxDrivingTime[327,508] : X[327,508] = 0
  [327, 509]  =  maxDrivingTim

> **Tip**
>
> Don’t confuse the `weightedDriving` matrix with the `drivingTimes`
> matrix! The `weightedDriving` matrix contains the weighted driving
> times, while the `drivingTimes` matrix contains the driving times.

Again, solve the model.

In [1]:
# YOUR CODE BELOW
optimize!(pMedianModel)

Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e-01, 4e+04]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+01]
Presolving model
89609 rows, 10821 cols, 109100 nonzeros  0s
21346 rows, 10786 cols, 69660 nonzeros  0s

Solving MIP model with:
   21346 rows
   10786 cols (10786 binary, 0 integer, 0 implied int., 0 continuous)
   69660 nonzeros
MIP-Timing:         0.2 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   379.33          inf       

The following code prints the model status and visualizes the districts.
If your implementation is correct, the districts with previously longer
driving times should have shrinked. Furthermore, the model should have
reached optimality or found a feasible solution before hitting the time
limit.

In [1]:
print_model_status(pMedianModel)
display(visualize_departments(hexshape, X, p))


Great, the solution is optimal.
The relative gap is 0.0
The solve time (in seconds) is 0.6825638748705387
The objective value is 341527.94

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2353.77,977.854 2382.21,925.951

# 4. Planning for Peak Demand

During major events or crime waves, districts need backup support from
neighboring stations. The Police Union has emphasized this as a critical
safety requirement for their officers. Your task: Design constraints
ensuring each district has backup support within 20 minutes of the
driving time.

In [1]:
# YOUR CODE BELOW
@constraint(pMedianModel, 
    support[i=rangeDepartments],
    sum(X[k,k] for k in rangeDepartments if drivingTimes[k,i] < 20 && k != i) >= X[i,i]
    )


1-dimensional DenseAxisArray{JuMP.ConstraintRef{JuMP.Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, JuMP.ScalarShape},1,...} with index sets:
    Dimension 1, [406, 365, 210, 439, 269, 504, 38, 255, 341, 155  …  188, 250, 9, 6, 69, 400, 141, 2, 496, 327]
And data, a 88-element Vector{JuMP.ConstraintRef{JuMP.Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, JuMP.ScalarShape}}:
 support[406] : X[317,317] + X[318,318] + X[349,349] + X[365,365] + X[367,367] + X[381,381] + X[382,382] + X[392,392] - X[406,406] + X[409,409] + X[412,412] + X[422,422] + X[425,425] + X[438,438] + X[439,439] + X[457,457] + X[466,466] + X[468,468] + X[496,496] + X[498,498] ≥ 0
 support[365] : X[262,262] + X[272,272] + X[288,288] + X[306,306] + X[308,308] + X[317,317] + X[318,318] + X[341,341] + X[349,349] + X[356,356] - X[365,365] + X[367,367] + X[368,368] +

> **Tip**
>
> This is a rather complicated task, so don’t worry if you cannot find
> the correct restriction. Try to come up with something or describe
> your thoughts in comments if you do not find the correct restriction
> or a way to implement it. If you are not sure, it often helps to start
> on paper and to draw the districts and the BAs to find a way to
> implement it.

In case you came up with a solution, solve the model to optimize the
layout of the districts.

In [1]:
# YOUR CODE BELOW
optimize!(pMedianModel)

Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e-01, 4e+04]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+01]
Presolving model
89697 rows, 10821 cols, 111006 nonzeros  0s
21422 rows, 10786 cols, 71370 nonzeros  0s

Solving MIP model with:
   21422 rows
   10786 cols (10786 binary, 0 integer, 0 implied int., 0 continuous)
   71370 nonzeros
MIP-Timing:        0.21 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   379.33          inf       

If your implementation is correct, the districts are contiguous and
distributed across the city. In addition, the model should have reached
optimality or found a feasible solution before hitting the time limit.

In [1]:
print_model_status(pMedianModel)
display(visualize_departments(hexshape, X, p))


Great, the solution is optimal.
The relative gap is 0.0
The solve time (in seconds) is 17.200854250229895
The objective value is 347573.89999999985

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2353.77,977.854 2382.21,925.951

------------------------------------------------------------------------

# Solutions

You will likely find solutions to most exercises online. However, I
strongly encourage you to work on these exercises independently without
searching explicitly for the exact answers to the exercises.
Understanding someone else’s solution is very different from developing
your own. Use the lecture notes and try to solve the exercises on your
own. This approach will significantly enhance your learning and
problem-solving skills.

Remember, the goal is not just to complete the exercises, but to
understand the concepts and improve your programming abilities. If you
encounter difficulties, review the lecture materials, experiment with
different approaches, and don’t hesitate to ask for clarification during
class discussions.

Later, you will find the solutions to these exercises online in the
associated GitHub repository, but we will also quickly go over them in
next week’s tutorial. To access the solutions, click on the Github
button on the lower right and search for the folder with today’s lecture
and tutorial. Alternatively, you can ask ChatGPT or Claude to explain
them to you. But please remember, the goal is not just to complete the
exercises, but to understand the concepts and improve your programming
abilities.